In [2]:
! pip install mysql

  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [24 lines of output]
      Trying pkg-config --exists mysqlclient
      Command 'pkg-config --exists mysqlclient' returned non-zero exit status 1.
      Trying pkg-config --exists mariadb
      Command 'pkg-config --exists mariadb' returned non-zero exit status 1.
      Trying pkg-config --exists libmariadb
      Command 'pkg-config --exists libmariadb' returned non-zero exit status 1.
      Traceback (most recent call last):
        File "/opt/conda/lib/python3.10/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 353, in <module>
          main()
        File "/opt/conda/lib/python3.10/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
        

In [3]:
!pip uninstall mysql -y

In [4]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 124.5 MB/s eta 0:00:0000:01


In [10]:
import pandas as pd
import pyarrow.parquet as pq
from google.cloud import storage
from google.oauth2 import service_account
import mysql.connector
from mysql.connector import Error
import io
from datetime import datetime

# GCS configuration
gcp_bucket_name = "myfirstproject-inputbucket"
gcp_credentials_path = "double-balm-438113-f5-53067ace2784.json"

# MySQL configuration
mysql_config = {
    'host': '34.95.229.146',
    'database': 'myindustry',
    'user': 'professor',
    'password': 'senha'
}

def get_gcs_client():
    credentials = service_account.Credentials.from_service_account_file(gcp_credentials_path)
    return storage.Client(credentials=credentials)

def read_parquet_from_gcs(bucket, blob_name):
    blob = bucket.blob(blob_name)
    buffer = io.BytesIO()
    blob.download_to_file(buffer)
    buffer.seek(0)
    return pq.read_table(buffer).to_pandas()

def insert_data_to_mysql(connection, df):
    cursor = connection.cursor()
    
    # Create table if not exists
    create_table_query = """
    CREATE TABLE IF NOT EXISTS event_data (
        day INT,
        month INT,
        year INT,
        tenantId VARCHAR(255),
        eventType VARCHAR(255),
        eventId VARCHAR(255),
        temperature FLOAT,
        relativeHumidity FLOAT,
        updateTime DATETIME,
        deviceId VARCHAR(255),
        projectId VARCHAR(255),
        deviceType VARCHAR(255),
        productNumber VARCHAR(255)
    )
    """
    cursor.execute(create_table_query)
    
    # Insert data
    insert_query = """
    INSERT INTO event_data (
        day, month, year, tenantId, eventType, eventId, 
        temperature, relativeHumidity, updateTime,
        deviceId, projectId, deviceType, productNumber
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    
    for _, row in df.iterrows():
        data = (
            row['day'], row['month'], row['year'],
            row['tenantId'], row['eventType'], row['eventId'],
            row['data']['event']['data']['humidity']['temperature'],
            row['data']['event']['data']['humidity']['relativeHumidity'],
            row['data']['event']['data']['humidity']['updateTime'],
            row['data']['metadata']['deviceId'],
            row['data']['metadata']['projectId'],
            row['data']['metadata']['deviceType'],
            row['data']['metadata']['productNumber']
        )
        cursor.execute(insert_query, data)
    
    connection.commit()
    print(f"Inserted {len(df)} rows into MySQL")

def main():
    storage_client = get_gcs_client()
    bucket = storage_client.bucket(gcp_bucket_name)
    
    connection = None
    try:
        connection = mysql.connector.connect(**mysql_config)
        if connection.is_connected():
            print("Connected to MySQL database")
            
            # List all blobs in the bucket
            blobs = bucket.list_blobs()
            
            for blob in blobs:
                if blob.name.endswith('.parquet'):
                    print(f"Processing file: {blob.name}")
                    df = read_parquet_from_gcs(bucket, blob.name)
                    insert_data_to_mysql(connection, df)
            
    except Error as e:
        print(f"Error while connecting to MySQL: {e}")
    finally:
        if connection is not None and connection.is_connected():
            connection.close()
            print("MySQL connection is closed")

if __name__ == "__main__":
    start_time = datetime.now()
    print(f"Start time: {start_time}")
    
    main()
    
    end_time = datetime.now()
    print(f"End time: {end_time}")
    print(f"Elapsed time: {end_time - start_time}")

Start time: 2024-10-18 09:41:42.589002
Connected to MySQL database
Processing file: 8j9f_c4hd_.parquet
Inserted 1 rows into MySQL
Processing file: i1qp_3mdc_.parquet
Inserted 1 rows into MySQL
Processing file: yk53_av8o_.parquet
Inserted 1 rows into MySQL
MySQL connection is closed
End time: 2024-10-18 09:41:45.237440
Elapsed time: 0:00:02.648438


In [8]:
!nc -zv 34.95.229.146 3306

nc: connect to 34.95.229.146 port 3306 (tcp) failed: Connection timed out


In [9]:
!curl ifconfig.me

34.95.176.129